# Redes Neurais

Redes Neurais é um dos algoritmos mais antigos de Machine Learning. Sia interpretação é foi baseada em um neurônio biológico e suas abstrações para o mundo digital. Possui um poder de mapeamento universal (Teorema de Aproximação Universal: Uma rede neural com 1 camada intermediária é capaz de aproximar qualquer mapeamento, dada algumas limitações). 

Desta forma, são um dos algoritmos mais poderosos no arsenal de um Cientista de Dados que trabalha com dados complexos. Semelhante ao Random Forest, o algoritmo deve ajustar uma quantidade gigantesca de parâmetros, o que pode ser considerado uma faca de dois gumes, pois se feito de forma errada, é bem provável que entre em regime de overfitting.

Neste notebook vamos estudar Redes Neurais Artificiais e Redes Neurais Convolucionais, onde vamos entender suas principais diferenças, e entender porquê existe essa hype tão grande em cima das Redes Neurais Profundas (Deep Learning).

In [ ]:
# Importando as bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importando o dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [ ]:
X.head()

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X.head()

In [ ]:
X.drop(['Geography_France', 'Gender_Male'], axis=1, inplace=True) # Evitando Dummy Variable Trap

In [ ]:
X.head()

In [ ]:
# Dividindo em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
pd.DataFrame(X_train).head()

## Criando a Rede Neural

In [ ]:
# Importando as bibliotecas e classes
import keras
from keras.models import Sequential
from keras.layers import Dense

# Inicializando a rede
classifier = Sequential()

# Adicionando a primeira camada escondida
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11)) # 6 neurônios, é a segunda camada escondida

# Adicionando a segunda camada escondida
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu')) # não é necessário informar a camada de saída

# Adicionando a camada de saída
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid')) # Uma saída, utilizando ativação sigmoid para probabilidades

# Compilando a rede
# adam = Um tipo de descida de gradiente estocástico
# metrics = accuracy
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Neste caso, nossa rede neural tem 3 camadas, com 11 neurônios na primeira camada, 6 neurônios na segunda camada, e 1 neurônio na cama de saída.

Nas camadas de Entrada e Escondida foram utilizadas funções de ativação ReLu, e na camada de saída foi utilizada a função de ativação Sigmoid (problema de classificação binária).

O algoritmo de otimização utilizado foi ADAM (um algoritmo tipo Descida de Gradiente Estocástico), e a métrica utilizada foi acurácia.

In [ ]:
# Treinando a rede
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

In [ ]:
# Prevendo os resultados
y_pred = classifier.predict(X_test) # Retorna probabilidades
y_pred = (y_pred > 0.5) # Retorna 1 para maior que 0,5 e 0 caso contrário
y_pred

In [ ]:
# Criando a matriz de confusão e acurácia
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt = 'd')
plt.show()
print('Acurácia: {}'.format(acc))

# Redes Neurais Convolucionais

Redes Neurais Convolucionais são amplamente utilizadas em reconhecimento de imagens, reconhecimento de voz, geração artificial de imagens e voz, no campo médico, entre outras aplicações. Seu poder está na imensa quantidade de pesos ajustados e em sua capacidade de extração automática de features, não sendo necessário nenhum preparo prévio de estudo de relevância de features (a rede aprende o que é importante e o que não é).

Consequentemente, dada a imensa quantidade de pesos à serem ajustados, o custo computacional de treino de uma rede neural convolucional é alto. Para estes casos, utiliza-se GPUs, ou Unidades Gráficas de Processamento, que aceleram o processamento desse tipo de Rede.

Para nosso curso, usaremos um conjunto de dados pequeno para que não seja necessário extrapolar as capacidades dos notebooks.

In [ ]:
# Importando as bibliotecas e Classes
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
# Inicializando a Rede
classifier = Sequential()

# 32, 3, 3 - Número de filtros, Dimensão dos filtros (3x3)
# input shape = 3 camadas (RGB), 64x64 pixels em cada
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

# 2x2 - Tamanho da forma de pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.4))

# Adicionando uma segunda camada de convolução e maxpooling
classifier.add(Convolution2D(32, 3, 3, activation = 'relu')) # não é necessário informar input_shape
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Flattening
classifier.add(Flatten())

# Full Connection
# Output_dim = Número de neurônios na camada escondida = 128
classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid')) # output layer, sigmoid function, binary outcome

# Compilando a CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Image augmentation - Usado para evitar overfitting
# Image augmentation cria um novo batch de imagens com pequenas mudanças, 
# de forma a aumentar virtualmente o número de imagens
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, # Cria imagens com tamanhos diferentes
                                   shear_range=0.2, # Transformações geométricas
                                   zoom_range=0.2, # Zoom randômico 
                                   horizontal_flip=True) # Flipping

test_datagen = ImageDataGenerator(rescale=1./255) # É apenas necessário fazer rescale

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64), # Dimensão esperada pela CNN
                                                 batch_size=32, # Número de imagens simultâneas na CNN
                                                 class_mode='binary') # 2 classes

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                             target_size=(64, 64),
                                             batch_size=32,
                                             class_mode='binary')

In [ ]:
# Treinando a rede
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000/32, # samples_per_epoch/batch_size
                         epochs = 10,
                         verbose = 0,
                         validation_data = test_set,
                         validation_steps = 2000/32)

In [ ]:
y_pred = classifier.predict(test_set)

In [ ]:
y_pred = classifier.predict_generator(test_set)
